# Introduction to folium

In [96]:
import pandas as pd
import folium
import requests
from bs4 import BeautifulSoup
import numpy as np

# Processing data

In [31]:
df = pd.read_csv('P3_GrantExport.csv', delimiter=';',index_col=0)
df.head()

,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
Project Number,,,,,,,,,,,,,,
1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN


Deal with a special case

In [32]:
df.loc[101005]

Project Title                            SELECTS 2003: Eidgenössische Wahlen 2003
Project Title English                        SELECTS 2003: Federal elections 2003
Responsible Applicant                                            Malaguerra Carlo
Funding Instrument                               Swiss Priority Programmes (SPPs)
Funding Instrument Hierarchy                                           Programmes
Institution                     Schweizerische Akademie der Geistes-und Sozial...
University                                              Forschungskommission SAGW
Discipline Number                                                           10202
Discipline Name                                                 Political science
Discipline Name Hierarchy       Human and Social Sciences;Sociology, social wo...
Start Date                                                             01.02.2003
End Date                                                               31.01.2004
Approved Amount 

Change from 'Forschungskommission SAGW' to 'Forschungskommission - SAGW'

In [33]:
df.loc[101005, 'University'] = 'Forschungskommission - SAGW'

### Select universities applicable

Not all of the universities in the columns `university` will be used. Some of the entries are not in Switzerland, and some of them didn't specifiy the universities. 

We will drop out some itmes:

- foreign universities where df['University'] is NaN
- 'Nicht zuteilbar' : not applicable
- 'NPO (Biblioth., Museen, Verwalt.) - NPO' : the exact places of NPOs are not known
- 'Weitere Institute - FINST' means other institutions
- 'Weitere Spitäler - ASPIT' means other universities
- 'Firmen/Privatwirtschaft - FP' means private/business
- 'Istituto Svizzero di Roma - ISR' is a Italian university

Besides, we will also negelect projects whose 'Approved Amount' is not given (`'data not included in P3'`).

In [97]:
foreign_institution = df['University'].isnull()
na                  = df['University'] == 'Nicht zuteilbar - NA'
npo                 = df['University'] == 'NPO (Biblioth., Museen, Verwalt.) - NPO'
other_institution   = df['University'] == 'Weitere Institute - FINST'
other_hospital      = df['University'] == 'Weitere Spitäler - ASPIT'
private             = df['University'] == 'Firmen/Privatwirtschaft - FP'
Italian_unvi        = df['University'] == 'Istituto Svizzero di Roma - ISR'

fund_amount_included = df['Approved Amount'] == 'data not included in P3'

swiss_univ_idx = df.index[(~foreign_institution) & (~npo) & (~fund_amount_included)
                         & (~na) & (~other_institution) & (~other_hospital)
                         & (~private) & (~Italian_unvi)]
swiss_df = df.loc[swiss_univ_idx]

Conver the money to float number

In [99]:
swiss_df['Approved Amount'] = swiss_df['Approved Amount'].astype(np.float64)

#### Group the dataframe by their university

In [100]:
univ_amount = swiss_df.groupby('University').sum()['Approved Amount']
univ_amount.sort_values(ascending=False).head(10)

University
Université de Genève - GE       1.838237e+09
Universität Zürich - ZH         1.826843e+09
ETH Zürich - ETHZ               1.635597e+09
Universität Bern - BE           1.519373e+09
Universität Basel - BS          1.352251e+09
Université de Lausanne - LA     1.183291e+09
EPF Lausanne - EPFL             1.175316e+09
Université de Fribourg - FR     4.575262e+08
Université de Neuchâtel - NE    3.832046e+08
Paul Scherrer Institut - PSI    1.152690e+08
Name: Approved Amount, dtype: float64

#### Extract universities and their abbreviations

In [38]:
splited_names = swiss_df.loc[swiss_univ_idx].University.apply(lambda x: x.split(' - '))
univ_name = splited_names.apply(lambda x: x[0])
univ_abb  = splited_names.apply(lambda x: x[1])
univ_name_unique = univ_name.unique()
univ_abb_unqiue  = univ_abb.unique()
name_to_univ = dict(zip(univ_name_unique, swiss_df.loc[swiss_univ_idx].University.unique()))

# Map university to their Canton

First, we use the api http://www.geonames.org/export/web-services.html to search the universities by their names. But not all of them can be found in this way.

Second, for those universities whose name are not listed in `geonames.com`, we manually substitute them with other keywords which will get a desirable result in `geonames.com`. For instance, we can search the name of university, the abbreviation of university, and even the city(street) of the unviersity.

As soon as we have obtained all the webpage results from `geonames.com`, we will parser the locations of universities. 
There are two kinds of possible results for each request:

    1. results found in database
    2. results found in wikipedia
    
Both of them contains `latitude` and `longitude` of a place, but the canton a university located can only be found in `database`. Thus we decide to use `reverse geocoding` to find the name of canton from the `latitude` and `longitude`.

As the number of requests for google maps is limited, we use `http://nominatim.openstreetmap.org/search.php` to for `reverse geocoding`. 

In [102]:
def request_geo_name(univ):
    original_url = 'http://www.geonames.org/search.html?'
    payload = {'q':univ,'country':'CH'}
    get_url = requests.get(original_url, params=payload, timeout=30)
    soup = BeautifulSoup(get_url.text,"lxml")
    return soup

def reverse_geocoding(lat, lng):
    reverse_geocoding_url = 'http://nominatim.openstreetmap.org/search.php'
    payload = {'q':lat+', '+lng}
    get_url = requests.get(reverse_geocoding_url, params=payload, timeout=30)
    soup = BeautifulSoup(get_url.text,"lxml")
    return ((soup.select('div[id=content] span[class=name]')[0].string).split(', ')[-6])

First, request with university names first

In [42]:
selectors = dict()
for name in univ_name_unique:
    soup = request_geo_name(name)
    table_selector = soup.select('div table[class=restable]')
    selectors[name] = table_selector

In [43]:
univ_found_by_name = [name for name in univ_name_unique if (selectors[name] != [])]
not_found_by_name = list(filter(lambda x: x not in univ_found_by_name, univ_name_unique))
print(len(univ_found_by_name))
print(len(not_found_by_name))

22
49


Modifiy the keywords in the request to get the accurate results.

In [104]:
modified_list = [
"WSL", "Graduate Institute of International and Development Studies", "Chur", "EPFL", 
"Zurich University of Applied Sciences/ZHAW", "Zürich", "University of Lugano edu",
"Bern", "St. Gallen", "Swiss Federal Institute of Aquatic Science and Technology",
"empa", "Davos", "Neuchâtel", "Davos", "University of Zürich", "frick", "agroscope",
"Bellinzona", "Epalinges", "Zollikofen", "Winterthur", "Chur", "FHNW", "Schaffhauserstrasse",
"Sion", "Bern", "Nottwil", "Avenue du Grand-Champsec", "HES-SO", "Chaux-de-Fonds", "HEP-BEJUNE",
"University of applied sciences in Lucerne", "Bern", "SUPSI", "Fribourg", "Arth", "Kreuzlingen",
"Kreuzlingen", "Zürich", "Lausanne", "Saint-Maurice valais", "Rorschach", "Zug", "Locarno",
"Schaffhausen", "Nordwestschweiz", "Basel", "Lugano","Brig"]
unmodified_to_modified = dict(zip(not_found_by_name, modified_list))

In [45]:
for name in not_found_by_name:
    modified_names = unmodified_to_modified[name]
    soup = request_geo_name(modified_names)
    table_selector = soup.select('div table[class=restable]')
    selectors[name] = table_selector

## Parse geo information

For two kinds of results, parse them seperately.

In [46]:
univ_loc = dict()
for name in univ_name_unique:
    found_in_wiki = 'wikipedia' in list((selectors[name][0].contents)[1].strings)
    if found_in_wiki:
        lat_lng = selectors[name][0].select('tr')[1].select('td[nowrap=] a')[-1].string
        univ_loc[name] = lat_lng.split(' / ')
    else: # found in database
        univ_loc[name] = [selectors[name][0].select('tr')[2].select('span[class=latitude]')[0].string,
                         selectors[name][0].select('tr')[2].select('span[class=longitude]')[0].string]

In [105]:
univ_loc

{'AO Research Institute': ['46.80428964656369', '9.837226867675781'],
 'Allergie- und Asthmaforschung': ['47.3765286779741', '8.54921936988831'],
 'Berner Fachhochschule': ['47.1044', '8.6747'],
 'Biotechnologie Institut Thurgau': ['47.650512', '9.175038'],
 "Centre de rech. sur l'environnement alpin": ['46.22739', '7.355592'],
 'EPF Lausanne': ['46.5193924550827', '6.56673431396484'],
 'ETH Zürich': ['47.3763034523898', '8.54804992675781'],
 'Eidg. Anstalt für Wasserversorgung': ['47.3983', '8.6194'],
 'Eidg. Forschungsanstalt für Wald,Schnee,Land': ['47.36044120058724',
  '8.454840045676514'],
 'Eidg. Hochschulinstitut für Berufsbildung': ['46.99904617562046',
  '7.458086013793945'],
 'Eidg. Material und Prüfungsanstalt': ['47.4031089613006',
  '8.61246419472767'],
 'Ente Ospedaliero Cantonale': ['46.19278438783632', '9.01702880859375'],
 'Fachhochschule Kalaidos': ['47.3666667', '8.55'],
 'Fachhochschule Nordwestschweiz (ohne PH)': ['47.5337767472998',
  '7.63856112957001'],
 'Fachh

## Build dataframe

Build a datadrame based on each university.

In [108]:
lat = []
lng = []
uname = []
amount = []
for name in univ_name_unique:
    lat.append(univ_loc[name][0])
    lng.append(univ_loc[name][1])
    uname.append(name)
    amount.append(univ_amount[name_to_univ[name]])

In [109]:
name_x_grant_x_geo = pd.DataFrame({'lat': lat, 'lng': lng, 'grant':amount },index=uname)

In [110]:
name_x_grant_x_geo.iloc[0]

grant    1.83824e+09
lat          46.1989
lng           6.1445
Name: Université de Genève, dtype: object

In [111]:
topo_path1 = r'ch-cantons.topojson.json'
ch_map = folium.Map(location=[46.8418522,7.9253733],
                   tiles='Mapbox Bright', zoom_start=8)
# ch_map.choropleth(geo_path=geo_path)
ch_map.choropleth(geo_path=topo_path1,
                  data = df_test,
                  columns=['id', 'v'],
                  key_on = "feature.id",
                  fill_color='YlGn',
                fill_opacity=0.3,
                line_weight=2,
                  topojson='objects.cantons')
ch_map

# Reverse geocoding

#### Parse the canton names from requested webpage, and map them to abbreviations

In [112]:
cantons = ["AG","AR","AI","BL","BS","BE","FR","GE","GL","GR",
 "JU","LU","NE","NW","OW","SG","SH","SZ","SO","TG","TI","UR","VS","VD","ZG","ZH"]

In [113]:
map_canton_name_to_abb = dict()
map_canton_name_to_abb['Genève'] = 'GE'
map_canton_name_to_abb['Basel-Stadt'] = 'BS'
map_canton_name_to_abb['Fribourg - Freiburg'] = 'FR'
map_canton_name_to_abb['Vaud'] = 'VD'
map_canton_name_to_abb['Bern - Berne'] = 'BE'
map_canton_name_to_abb['Neuchâtel'] = 'NE'
map_canton_name_to_abb['Sankt Gallen'] = 'SG'
map_canton_name_to_abb['Graubünden - Grigioni - Grischun'] = 'GR'
map_canton_name_to_abb['Luzern'] = 'LU'
map_canton_name_to_abb['Ticino'] = 'TI'
map_canton_name_to_abb['Jura'] = 'JU'
map_canton_name_to_abb['Schwyz'] = 'SZ'
map_canton_name_to_abb['Aargau'] = 'AG'
map_canton_name_to_abb['Basel-Landschaft'] = 'BL'
map_canton_name_to_abb['Valais - Wallis'] = 'VS'
map_canton_name_to_abb['Thurgau'] = 'TG'
map_canton_name_to_abb['Schaffhausen'] = 'SH'
map_canton_name_to_abb['Zug'] = 'ZG'
map_canton_name_to_abb['Zürich'] = 'ZH'

Requesting canton names by their positions

In [114]:
unvi_canton = []
for n in range(name_x_grant_x_geo.shape[0]):
    unvi_canton.append(reverse_geocoding(name_x_grant_x_geo.iloc[n]['lat'], name_x_grant_x_geo.iloc[n]['lng']))

In [116]:
unvi_canton[0:5]

['Genève', 'Basel-Stadt', 'Fribourg - Freiburg', 'Zürich', 'Vaud']

Map the name of university to their cantons

In [118]:
univ_canton_abb = [map_canton_name_to_abb[place] for place in unvi_canton]

Add a column to our dataframe called 'id' which coincides with those in topojson files

In [119]:
name_x_grant_x_geo['id'] = univ_canton_abb

Group the dataframe by their ids (cantons) and conver the unit to million francs

In [142]:
series = name_x_grant_x_geo.groupby('id').sum().reset_index()
series.grant = series.grant/10**6
series

,id,grant
0,AG,122.711379
1,BE,1524.119575
2,BL,46.248056
3,BS,1404.081126
4,FR,492.189387
5,GE,1879.078880
6,GR,17.368351
7,JU,35.593651
8,LU,54.673287
9,NE,402.525027


# Plot

Plot a map which combines the information from dataframe and topojson of Switzerland.

The color of map reflects the amount of grants approved in each canton (in million francs).

The location of each university is also marked in the map. Note that the `marker_cluster` we used will automatically 
aggragate the number of universities when zoomed.

In [160]:
topo_path1 = r'ch-cantons.topojson.json'
ch_map = folium.Map(location=[46.8418522,7.9253733], tiles='Mapbox Bright', zoom_start=8)
ch_map.choropleth(geo_path=topo_path1,
                  data = series,
                  columns=['id', 'grant'],
                  key_on = "feature.id",
                  fill_color='YlGn',
                  fill_opacity=0.3,
                  line_weight=2,
                  threshold_scale=[600, 1200, 1800, 2400, 3000, 3600],
                  legend_name='Grants approved (million francs)',
                  topojson='objects.cantons')

In [161]:
marker_cluster = folium.MarkerCluster().add_to(ch_map)

for i in range(len(uname)):
    folium.Marker(
        location=[float(lat[i]), float(lng[i])],
        popup=uname[i],
        icon=folium.Icon(color='red', icon='ok-sign'),
    ).add_to(marker_cluster)

ch_map